### Libraries

In [7]:
import math
import numpy as np
import random

import pandas 
import plotly.express as px

### Constants

In [8]:
# Gauss-Laguerre constants
# Notation: (abscissa, weight)
N = 6

GAUSS_LAGUERRE = [
    (0.222847, 0.458964),
    (1.188932, 0.417),
    (2.992736, 0.113373),
    (5.775144, 0.0103992),
    (9.837467, 0.000261017),
    (15.982874, 0.000000898548)
]

### Functions

In [9]:
# Weibull distribution function
def f_Weibull (x, k, mu):
    varLambdaDenom = lambda s, k : math.pow (s, 1/k)
    
    area = 0
    for i in range(N):
        area += GAUSS_LAGUERRE[i][1] * varLambdaDenom(GAUSS_LAGUERRE[i][0], k)
    
    varLambda = mu / area

    f = (k / varLambda) * math.pow(x / varLambda, k-1) * math.pow(math.e, -math.pow(x / varLambda, k))

    return f

In [10]:
# Survivability function S(t)
def cdf_Weibull (t, k, mu):
    area = 0
    
    for i in range(N):
        area += GAUSS_LAGUERRE[i][1] * f_Weibull( -GAUSS_LAGUERRE[i][0] + t, k, mu)

    return area

In [11]:
# Inverse survivability -- solved as root-finding problem using 
def inverse_survivability(a, b, k, mu, alpha):
    f = lambda x : cdf_Weibull(x, k, mu) - alpha

    f_a = f(a)
    f_b = f(b)
    tol = 1.0e-9

    if f_a == 0.0:
        return a
    
    if f_b == 0.0:
        return b
    
    n = int( math.ceil( math.log(abs(b-a) / tol) / math.log(2.0) ) )

    for i in range(n):
        c = 0.5 * (a + b)
        f_c = f(c)

        if f_c == 0.0:
            return c

        if np.sign(f_a) == np.sign(f_c):
            a = c
            f_a = f_c
        
        elif np.sign(f_b) == np.sign(f_c):
            b = c
            f_b = f_c

    return 0.5 * (a+b)

### Evaluation

In [12]:
# Mu; average life expectancy
avg_life_expectancy = 78

# List for values
y_CDF_decrease = []
y_CDF_constant = []
y_CDF_increase = []
x_values = []

for x in range(0, 100, 1):
    x /= 100

    area_decrease = 0
    area_constant = 0
    area_increase = 0

    x_values.append(x)